# 파인튜닝 모델 추론 및 테스트

학습된 모델을 사용하여 용어 정의를 생성합니다.

---

## 1. 환경 설정

In [ ]:
%%capture
!pip install transformers peft accelerate

In [ ]:
import torch
from pathlib import Path
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import PeftModel

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

MODEL_PATH = Path('/content/drive/MyDrive/LLM_FineTuning_Project/model/')

## 2. 모델 로드

두 가지 방법 중 선택:
- **방법 1**: LoRA 어댑터만 로드 (메모리 효율적)
- **방법 2**: 병합된 전체 모델 로드 (더 간단)

In [ ]:
# LoRA 어댑터 로드
BASE_MODEL = "skt/kogpt2-base-v2"
ADAPTER_PATH = MODEL_PATH / 'lora_adapter'

# 베이스 모델 로드
base_model = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL,
    torch_dtype=torch.float16,
    device_map='auto'
)

# LoRA 어댑터 적용
model = PeftModel.from_pretrained(base_model, ADAPTER_PATH)
model = model.merge_and_unload() 

# 토크나이저 로드
tokenizer = AutoTokenizer.from_pretrained(ADAPTER_PATH)

print("모델 로드 완료!")

In [ ]:
#방법 2: 병합된 모델 로드
"""
MERGED_MODEL_PATH = MODEL_PATH / 'merged_model'

model = AutoModelForCausalLM.from_pretrained(
     MERGED_MODEL_PATH,
     torch_dtype=torch.float16,
     device_map='auto'
 )
tokenizer = AutoTokenizer.from_pretrained(MERGED_MODEL_PATH)

print("병합 모델 로드 완료!")
"""

## 3. 추론 함수 정의

In [ ]:
def generate_definition(term, domain="게임", facet=None, max_length=256):
    """
    용어의 정의를 생성합니다.

    Args:
        term: 정의할 용어
        domain: 분야 (게임, 레저, 미디어 등)
        facet: 세부 카테고리 (선택사항)
        max_length: 최대 생성 길이

    Returns:
        생성된 정의 문자열
    """
    # 프롬프트 생성
    if facet:
        instruction = f"다음은 {domain} 분야의 {facet} 관련 용어입니다. '{term}'의 정의를 설명해주세요."
    else:
        instruction = f"다음은 {domain} 분야의 용어입니다. '{term}'의 정의를 설명해주세요."

    prompt = f"""### 질문:
{instruction}

### 답변:
"""

    # 토큰화
    inputs = tokenizer(prompt, return_tensors='pt').to(device)

    # 생성
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=max_length,
            do_sample=True,
            temperature=0.7,
            top_p=0.9,
            top_k=50,
            repetition_penalty=1.2,
            pad_token_id=tokenizer.pad_token_id,
            eos_token_id=tokenizer.eos_token_id,
        )

    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # 답변 부분만 추출
    if "### 답변:" in generated_text:
        answer = generated_text.split("### 답변:")[1].strip()
    else:
        answer = generated_text[len(prompt):].strip()

    return answer

print("추론 함수 정의 완료!")

## 4. 테스트

In [ ]:
# 테스트할 용어 목록
test_terms = [
    {"term": "보스 몬스터", "domain": "게임", "facet": "몬스터"},
    {"term": "파밍", "domain": "게임", "facet": None},
    {"term": "던전", "domain": "게임", "facet": "지역"},
    {"term": "PvP", "domain": "게임", "facet": None},
    {"term": "버프", "domain": "게임", "facet": "스킬"},
]

print("=" * 60)
print("용어 정의 생성 테스트")
print("=" * 60)

for item in test_terms:
    print(f"\n용어: {item['term']}")
    print(f"   분야: {item['domain']}" + (f" / {item['facet']}" if item['facet'] else ""))
    print("-" * 40)

    definition = generate_definition(
        term=item['term'],
        domain=item['domain'],
        facet=item['facet']
    )

    print(f"   정의: {definition}")
    print()

## 5. 인터랙티브 테스트

In [ ]:
term = "탱커"  # 원하는 용어로 변경
domain = "게임"  # 게임, 레저, 미디어 중 선택
facet = "직업"  # 선택사항

print(f"\n'{term}' 정의 생성 중...\n")
result = generate_definition(term, domain, facet)
print(f"정의: {result}")

## 6. 성능 평가 필수x 선택

In [ ]:
import json
from pathlib import Path

# 검증 데이터로 평가
DATA_PATH = Path('/content/drive/MyDrive/LLM_FineTuning_Project/processed/')

def load_jsonl(file_path):
    data = []
    with open(file_path, 'r', encoding='utf-8') as f:
        for line in f:
            data.append(json.loads(line))
    return data

val_data = load_jsonl(DATA_PATH / 'val.jsonl')
print(f"검증 데이터 수: {len(val_data)}")

In [ ]:
import random

sample_indices = random.sample(range(len(val_data)), 5)

print("=" * 70)
print("정답 vs 생성 결과 비교")
print("=" * 70)

for idx in sample_indices:
    item = val_data[idx]

    # 생성
    generated = generate_definition(
        term=item['term'],
        domain=item['domain'],
        facet=item.get('facet')
    )

    print(f"\n용어: {item['term']}")
    print(f"\n[정답]")
    print(f"{item['output'][:200]}..." if len(item['output']) > 200 else item['output'])
    print(f"\n[생성]")
    print(f"{generated[:200]}..." if len(generated) > 200 else generated)
    print("-" * 70)